<a href="https://colab.research.google.com/github/sheelapravalika/AI_CS_mokshasaireddy005/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U datasets huggingface_hub fsspec pandas scikit-learn matplotlib seaborn numpy

# ✅ STEP 2: Import Libraries
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)


In [ ]:
# ✅ STEP 3: Load CIC-IDS2017 (Binary) Dataset
from datasets import load_dataset
dataset = load_dataset("sonnh-tech1/cic-ids-2017", "binary")
df = dataset["train"].to_pandas()
print(df)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/92.8M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2522362 [00:00<?, ? examples/s]

         DestinationPort  FlowDuration  TotalFwdPackets  TotalBackwardPackets  \
0                  54865             3                2                     0   
1                  55054           109                1                     1   
2                  55055            52                1                     1   
3                  46236            34                1                     1   
4                  54863             3                2                     0   
...                  ...           ...              ...                   ...   
2522357               53         32215                4                     2   
2522358               53           324                2                     2   
2522359            58030            82                2                     1   
2522360               53       1048635                6                     2   
2522361               53         94939                4                     2   

         TotalLengthofFwdPa

In [ ]:
# ✅ STEP 4: Preprocess Data
#df.dropna(inplace=True)  # Drop missing rows
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])  # Benign=0, Threat=1
X = df.drop('Label', axis=1)
y = df['Label']
print(X,y)
X_numeric = X.select_dtypes(include=['int64', 'float64'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)
print(X_scaled)
print(y)

         DestinationPort  FlowDuration  TotalFwdPackets  TotalBackwardPackets  \
0                  54865             3                2                     0   
1                  55054           109                1                     1   
2                  55055            52                1                     1   
3                  46236            34                1                     1   
4                  54863             3                2                     0   
...                  ...           ...              ...                   ...   
2522357               53         32215                4                     2   
2522358               53           324                2                     2   
2522359            58030            82                2                     1   
2522360               53       1048635                6                     2   
2522361               53         94939                4                     2   

         TotalLengthofFwdPa

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import pandas as pd

# Define the parameter grid
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create the classifier
rf = RandomForestClassifier(random_state=42)

# Randomized Search CV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,  # Number of random combinations to try
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring='accuracy'  # or 'f1_weighted' for imbalanced classes
)

# Fit on training data
random_search.fit(X_train, y_train)
results_df = pd.DataFrame(random_search.cv_results_)


top_results = results_df.sort_values(by='mean_test_score', ascending=False)
print(top_results[['mean_test_score', 'std_test_score', 'params']].head(10))

Fitting 3 folds for each of 30 candidates, totalling 90 fits
    mean_test_score  std_test_score  \
12         0.998525        0.000022   
20         0.998512        0.000014   
10         0.998511        0.000022   
17         0.998511        0.000010   
13         0.998455        0.000015   
8          0.998439        0.000018   
21         0.998439        0.000035   
22         0.998434        0.000024   
23         0.998401        0.000021   
25         0.998398        0.000044   

                                               params  
12  {'n_estimators': 100, 'min_samples_split': 5, ...  
20  {'n_estimators': 200, 'min_samples_split': 5, ...  
10  {'n_estimators': 100, 'min_samples_split': 2, ...  
17  {'n_estimators': 100, 'min_samples_split': 5, ...  
13  {'n_estimators': 200, 'min_samples_split': 5, ...  
8   {'n_estimators': 200, 'min_samples_split': 10,...  
21  {'n_estimators': 100, 'min_samples_split': 2, ...  
22  {'n_estimators': 200, 'min_samples_split': 10,...  
23  {

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Assuming X_scaled and y are defined in previous cells and are available.
# If not, you may need to include the data loading and preprocessing steps here as well.

# Re-perform train/test split to ensure X_test and y_test are defined
# Using the same parameters as in the model training/tuning cell for consistency
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the parameter grid (same as in the previous tuning cell)
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Create the classifier (same as in the previous tuning cell)
rf = RandomForestClassifier(random_state=42)

# Randomized Search CV (included here to ensure 'random_search' is defined)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,  # Number of random combinations to try
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring='accuracy'
)

# Fit on training data (included here to ensure 'random_search' is fitted)
random_search.fit(X_train, y_train)


# ✅ 1. Get the best model from Randomized Search
best_rf = random_search.best_estimator_

# ✅ 2. Predict on the test set
y_pred = best_rf.predict(X_test)

# ✅ 3. Accuracy & Classification Report
print(f"\n✅ Accuracy on Test Set: {accuracy_score(y_test, y_pred):.4f}")
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred, digits=4))

# ✅ 4. Confusion Matrix (raw)
print("\n📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Fitting 3 folds for each of 30 candidates, totalling 90 fits

✅ Accuracy on Test Set: 0.9986

📄 Classification Report:
               precision    recall  f1-score   support

           0     0.9993    0.9993    0.9993    419148
           1     0.8892    0.6973    0.7816       403
           2     1.0000    0.9996    0.9998     25498
           3     0.9990    0.9941    0.9965      2032
           4     0.9990    0.9966    0.9978     34853
           5     0.9894    0.9904    0.9899      1038
           6     0.9954    0.9880    0.9917      1085
           7     1.0000    0.9983    0.9991      1160
           8     1.0000    1.0000    1.0000         2
           9     1.0000    0.7500    0.8571         4
          10     0.9888    0.9987    0.9937     18187
          11     1.0000    0.9905    0.9952       633
          12     0.7209    0.8435    0.7774       294
          13     0.0000    0.0000    0.0000         1
          14     0.4884    0.3111    0.3801       135

    accuracy  